# Walmart Stores Sales Forecasting
**Kaan Akkartal**

# 1. Library and Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.linear_model import LinearRegression
from sklearn import tree

In [ ]:
features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')

# 2. Data Observation

Four different tables are added. When we look at the beginning and end, we observe that it begins with 2010-02-05 and ends with 2013-07-26.

In [ ]:
features.head()

In [ ]:
features.tail()

In [ ]:
features.shape

In [ ]:
features.info()

There are missing values in Markdown, CPI and Unemployment variables.

Below the amount of missing values are calculated. More than half of the observations are missing in Markdown variables. 

In [ ]:
features.isnull().sum()

In [ ]:
features.describe()

As in "Features" table, "Train" table starts with 2010-02-05 but it ends with 2012-10-26. The amount of sales and holiday information stand on the table. In addition there are store and departmant values. 

In [ ]:
train.head()

In [ ]:
train.tail()

There are no missing values in "Train" table.

In [ ]:
train.info()

"Test" table begins where "Train" table ends in terms of date. So it starts with "2012-11-02". 

In [ ]:
test.head()

In [ ]:
test.tail()

No missing values in "test" table. 

In [ ]:
test.info()

In [ ]:
stores.head()

In [ ]:
stores.tail()

No missing values in "stores" table.

In [ ]:
stores.info()

# 3. Data Manipulation

In [ ]:
features

In [ ]:
train

In [ ]:
df = pd.merge(train, features, how = "left")
df

In [ ]:
df = pd.merge(df, stores, how = "left")
df

In [ ]:
df.isnull().sum()

In [ ]:
df.Date = pd.to_datetime(df.Date)

In [ ]:
df.set_index(keys = "Date", inplace = True)

In [ ]:
df.head()

In [ ]:
df.index

Markdown variables are dropped becuase more than half of the observations are missing.

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(axis = 1, columns = ["MarkDown1", "MarkDown2","MarkDown3","MarkDown4", "MarkDown5"], inplace = True)
df.head()

In [ ]:
df.Store = pd.Categorical(df.Store)
df.Dept = pd.Categorical(df.Dept)

In [ ]:
df.dtypes

In [ ]:
df.describe()

# 4. Data Visualization

In [ ]:
ax = sns.scatterplot(x = df.index , y ="Weekly_Sales", data = df)
ax.set_xlim(df.index.min(), df.index.max())
for item in ax.get_xticklabels():
    item.set_rotation(90);

In [ ]:
sns.boxplot(x = "Weekly_Sales", data = df);

In [ ]:
df.Weekly_Sales[df.Weekly_Sales == max(df.Weekly_Sales)]

Top 5 sales take place in Thank's Giving week.

In [ ]:
top5_sales = list(df.Weekly_Sales.sort_values(ascending=False).head(5).values)
top5_sales

In [ ]:
df.loc[df.Weekly_Sales.isin(top5_sales)].sort_values(by = "Weekly_Sales", ascending = False)

Boxplotting for each variable. 

In [ ]:
df[['Store', 'Dept', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Type', 'Size']].plot(kind='box',subplots=1,layout=(3,5),figsize=(14,12))

Because of the possible outlier values "Weekly_Sales" and "Unemployment" variables are observed. Unlike "Weekly_Sales", "Unemployment" values are close to normal distribution. 

In [ ]:
plt.xlim(0, 150000)
rcParams['figure.figsize'] = 1,6.0
sns.distplot(df.Weekly_Sales, kde = False);

In [ ]:
rcParams['figure.figsize'] = 5,6.0
sns.distplot(df.Unemployment, kde = False);

In [ ]:
rcParams['figure.figsize'] = 14,5
sns.lineplot(x = df.index, y = "Weekly_Sales", data = df);

Sales are increased dramaticly on year ends. Christmas and Thank's Giving affect those values directly.

### **Barplot and lineplot diagrams for each variable**

In [ ]:
rcParams['figure.figsize'] = 16.0,6.0
sns.barplot(x = "Store", y = "Weekly_Sales", data = df);

In [ ]:
rcParams['figure.figsize'] = 16.0,6.0
sns.barplot(x = "Dept", y = "Weekly_Sales", data = df);

In [ ]:
rcParams['figure.figsize'] = 10,6.0
sns.lineplot(x = "Fuel_Price", y = "Weekly_Sales", data = df);

In [ ]:
rcParams['figure.figsize'] = 10,6.0
sns.lineplot(x = "CPI", y = "Weekly_Sales", data = df);

In [ ]:
rcParams['figure.figsize'] = 10,6.0
sns.lineplot(x = "Unemployment", y = "Weekly_Sales", data = df);

In [ ]:
rcParams['figure.figsize'] = 16,6.0
sns.lineplot(x = df.index, y = "Weekly_Sales", hue = "Type", data = df);

Sales rate by store type: (A>B>C). 

In [ ]:
sns.boxplot(x = "Type", y = "Size", data = df);

Store size is also aligned with A>B>C. So it is correlated to sales rate.

In [ ]:
sns.lineplot(x = "Size" , y = "Weekly_Sales", hue = "Type", data = df);

In [ ]:
rcParams['figure.figsize'] = 10,6.0
sns.lineplot(x = "Temperature" , y = "Weekly_Sales", data = df);

In [ ]:
sns.lineplot(x = df.index.month, y = "Weekly_Sales", data = df);

In [ ]:
sns.lineplot(x = df.index.month, y = "Weekly_Sales", hue = "IsHoliday", data = df);

# 5. Correlation

In [ ]:
sns.heatmap(df.corr(), vmin = -1, vmax = 1, annot = True);

In [ ]:
df.dtypes

In [ ]:
df.IsHoliday = pd.Categorical(df.IsHoliday)
df.Type = pd.Categorical(df.Type)

In [ ]:
df.dtypes

# 6. Modelling

### Model Preparation

In [ ]:
df

In [ ]:
df_X = df.drop("Weekly_Sales", axis = 1)
df_X

In [ ]:
df_Y = df.Weekly_Sales
df_Y

In [ ]:
df_X = pd.get_dummies(df_X, drop_first = True)

In [ ]:
df_X.reset_index(inplace = True)

In [ ]:
df_X = df_X.iloc[:,1:]
df_X

In [ ]:
from sklearn.model_selection import train_test_split
df_X_train, df_X_validation, df_Y_train, df_Y_validation = train_test_split(df_X , df_Y, test_size = 0.2)

In [ ]:
df_X_train.head()

In [ ]:
df_X_validation

### ML Algorithms 

**LINEAR REGRESSION**

In [ ]:
lm = LinearRegression()

In [ ]:
model = lm.fit(df_X_train, df_Y_train)

In [ ]:
lm.score(df_X_validation, df_Y_validation)

In [ ]:
lm.score(df_X_train, df_Y_train)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [ ]:
cross_val_score(model, df_X_train, df_Y_train, cv = 10, scoring = "r2").mean()

In [ ]:
np.sqrt(-cross_val_score(model, 
                df_X_train, 
                df_Y_train, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()

In [ ]:
y_pred = lm.predict(df_X_validation)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
mse = mean_squared_error(df_Y_validation, y_pred)
mse

In [ ]:
rmse = np.sqrt(mse)
rmse

In [ ]:
df_Y.values.mean()

**DECISION TREE**

In [ ]:
clf = tree.DecisionTreeRegressor()

In [ ]:
model = clf.fit(df_X_train, df_Y_train)

In [ ]:
clf.score(df_X_validation, df_Y_validation)

In [ ]:
cross_val_score(model, df_X_train, df_Y_train, cv = 10, scoring = "r2").mean()

In [ ]:
y_pred = clf.predict(df_X_validation)

In [ ]:
mse = mean_squared_error(df_Y_validation, y_pred)
mse

In [ ]:
rmse = np.sqrt(mse)
rmse

 **XGBoost**

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model = XGBRegressor().fit(df_X_train, df_Y_train)

In [ ]:
xgb_model.score(df_X_validation, df_Y_validation)

In [ ]:
y_pred = xgb_model.predict(df_X_validation)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.sqrt(mean_squared_error(df_Y_validation, y_pred))

 **Light GBM**

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(df_X_train, df_Y_train)

In [ ]:
lgbm_model.score(df_X_validation, df_Y_validation)

In [ ]:
y_pred = lgbm_model.predict(df_X_validation)

In [ ]:
rmse = np.sqrt(mean_squared_error(df_Y_validation, y_pred))
rmse

**Model Validation and Tuning**

In [ ]:
lgbm_grid = {'n_estimators': [20, 40, 100, 200, 500, 1000],
             'learning_rate': [0.1, 0.01, 0.5]}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
lgbm = LGBMRegressor()
lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=5, n_jobs = -1, verbose = 2)

In [ ]:
lgbm_cv_model.fit(df_X_train, df_Y_train)

In [ ]:
lgbm_cv_model.best_params_

In [ ]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.5, n_estimators = 1000)

lgbm_tuned = lgbm_tuned.fit(df_X_train, df_Y_train)

In [ ]:
y_pred = lgbm_tuned.predict(df_X_validation)

In [ ]:
np.sqrt(mean_squared_error(df_Y_validation, y_pred))

In [ ]:
lgbm_tuned.score(df_X_validation, df_Y_validation)

Logistic Regression, Decision Tree, XGBoost and Light GBM models are applied. Best rmse value observed in tuned Light GBM model (4645). R2 is 96%. Therefore, final prediction will be done with tuned Light GBM model.

### Prediction

"Test", "Features" and "Stores tables are merged. 

In [ ]:
test.head()

In [ ]:
features.head()

In [ ]:
stores.head()

In [ ]:
df_test = pd.merge(test, features, how = "inner")
df_test

In [ ]:
df_test = pd.merge(df_test, stores, how = "inner")
df_test

In [ ]:
df_test.info()

Markdown variables were not in the model, so they are also dropped in "test" table. 

In [ ]:
df_test.drop(axis = 1, columns = ["MarkDown1", "MarkDown2","MarkDown3","MarkDown4", "MarkDown5"], inplace = True)
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.Date = pd.to_datetime(df_test.Date)
df_test.set_index(keys = "Date", inplace = True)
df_test.head()

In [ ]:
df_test.IsHoliday = pd.Categorical(df_test.IsHoliday)
df_test.Store = pd.Categorical(df_test.Store)
df_test.Dept = pd.Categorical(df_test.Dept)
df_test.Type = pd.Categorical(df_test.Type)

In [ ]:
df_test.dtypes

In [ ]:
df_test.describe()

In [ ]:
(115064-76902)/115064

Missing value rate of CPI and Unemployment variables are too much. Since they are included in the model, the missing observations are deleted.  

In [ ]:
df_test

In [ ]:
df_test.dropna(inplace = True)

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
dftest_X = df_test 

In [ ]:
dftest_X.head()

In [ ]:
dftest_X.reset_index(inplace = True)

In [ ]:
dftest_X.head()

In [ ]:
dftest_X = dftest_X.iloc[:,1:]

In [ ]:
dftest_X.head()

In [ ]:
dftest_X = pd.get_dummies(dftest_X, drop_first = True)

In [ ]:
predictions = lgbm_tuned.predict(dftest_X)
predictions

In [ ]:
predictions.dtype

In [ ]:
predictions = pd.Series(predictions)

In [ ]:
predictions.describe()

In [ ]:
df_test

In [ ]:
df_test["Prediction_Sales"] = predictions

In [ ]:
df_test

In [ ]:
df_test.describe()

In [ ]:
sns.lineplot(x = df_test.Date, y = "Prediction_Sales", data = df_test);